In [1]:
from pprint import pprint

print('Hello, World!')

Hello, World!


In [ ]:
from datetime import datetime
import requests

import ta
import pandas as pd

from lumibot.strategies import Strategy
from lumibot.backtesting import PandasDataBacktesting
from lumibot.entities import Asset, Bars, Data

base_asset = 'ETH'
quote_asset = 'USDC'
resolution = '5'  # 5 minute

class HydraStrategy(Strategy):
    parameters = {
        'fast_ema_period': 5,
        'slow_ema_period': 20,
    }

    def initialize(self):
        self.set_market('24/7')

    def on_trading_iteration(self):
        self.base = Asset(base_asset, Asset.AssetType.CRYPTO)
        self.quote = Asset(quote_asset, Asset.AssetType.CRYPTO)
        # print(f'Portfolio value: {self.portfolio_value}')

        # If cash exceeds initial budget by more than 10%, log a warning that target has been reached
        if self.cash >= self.initial_budget + (self.initial_budget * 0.1):
            print(f"Porfolio value {self.cash} exceeds initial budget {self.initial_budget} by more than 10%.")
            self.close_positions([self.base])
            raise Exception("Target reached, stopping strategy.")
        elif self.cash <= self.initial_budget - (self.initial_budget * 0.5):
            print(f"Porfolio value {self.cash} below initial budget {self.initial_budget} by more than 10%.")
            self.close_positions([self.base])
            raise Exception("Stop loss reached, stopping strategy.")
        else:
            bars: Bars = self.get_historical_prices(
                asset=self.base,
                quote=self.quote,
                timestep='5 minutes',
                length=100,
                timeshift=100 if self.first_iteration else 0
            )
            # print(bars.df)
            # raise Exception("Stopping after one iteration for demonstration purposes.")
            fast_ema = ta.trend.EMAIndicator(bars.df['close'], self.parameters["fast_ema_period"])
            slow_ema = ta.trend.EMAIndicator(bars.df['close'], self.parameters["slow_ema_period"])
            # print(f"Fast EMA: {fast_ema.ema_indicator()}")
            # raise Exception("Stopping after one iteration for demonstration purposes.")

            if fast_ema.ema_indicator().iloc[-1] > slow_ema.ema_indicator().iloc[-1]:
                # print("Golden cross detected!")
                if not self.get_position(self.base):
                    # print("Placing buy order!")
                    order = self.create_order(
                        asset=self.base,
                        quote=self.quote,
                        quantity=100,
                        side='buy'
                    )
                    self.submit_order(order)
                    if not self.is_backtesting: self.wait_for_order_execution(order)
            elif fast_ema.ema_indicator().iloc[-1] < slow_ema.ema_indicator().iloc[-1]:
                # print("Death cross detected!")
                if self.get_position(self.base):
                    # print("Placing sell order!")
                    order = self.create_order(
                        asset=self.base,
                        quote=self.quote,
                        quantity=100,
                        side='sell'
                    )
                    self.submit_order(order)
                    if not self.is_backtesting: self.wait_for_order_execution(order)


strategy = HydraStrategy()

backtesting_start = datetime(2025, 8, 29)
backtesting_end = datetime(2025, 8, 30)

url = "https://api.orderly.org/v1/tv/history"
querystring = {
    "symbol": f"PERP_{base_asset}_{quote_asset}",
    "resolution": resolution,
    "from": str(int(backtesting_start.timestamp())),
    "to": str(int(backtesting_end.timestamp())),
}
response = requests.get(url, params=querystring)
data = response.json()
df = pd.DataFrame({
    'datetime': data.get('t', []),
    'open': data.get('o', []),
    'high': data.get('h', []),
    'low': data.get('l', []),
    'close': data.get('c', []),
    'volume': data.get('v', [])
})
df['datetime'] = pd.to_datetime(df['datetime'], unit='s')
df.set_index('datetime', inplace=True)
asset = Asset(base_asset, Asset.AssetType.CRYPTO)
quote = Asset(quote_asset, Asset.AssetType.CRYPTO)
pandas_data = {
    asset: Data(
        asset,
        df,
        timestep='minute',
        quote=quote,
    )
}
# print(df.tail())

strategy.backtest(
    PandasDataBacktesting,
    backtesting_start=pandas_data[asset].datetime_start,
    backtesting_end=pandas_data[asset].datetime_end,
    budget=2000000,
    pandas_data=pandas_data,
    benchmark_asset=asset,
    quote_asset=quote,
)

                              open     high      low    close    volume
datetime                                                               
2025-08-29 22:40:00-04:00  4354.16  4355.00  4349.58  4351.06   72.2179
2025-08-29 22:45:00-04:00  4350.73  4354.92  4348.42  4349.64   33.8783
2025-08-29 22:50:00-04:00  4348.82  4358.95  4348.81  4358.01  145.5938
2025-08-29 22:55:00-04:00  4357.44  4366.55  4354.85  4366.55  201.2567
2025-08-29 23:00:00-04:00  4366.36  4370.55  4362.38  4362.38  188.2721
Progress || 100.00%  [Elapsed: 0:00:44 ETA: 0:00:00] Portfolio Val: 2,009,246.00

/snap/core20/current/lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.38' not found (required by /lib64/libduktape.so.207)
Failed to load module: /home/balo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.


[6994:6994:0100/000000.690579:ERROR:content/zygote/zygote_linux.cc:660] write: Broken pipe (32)


{'cagr': np.float64(5.339345576634073),
 'volatility': np.float64(0.09776251278472002),
 'sharpe': np.float64(54.20191670083334),
 'max_drawdown': {'drawdown': np.float64(0.002039328700624316),
  'date': Timestamp('2025-08-29 16:00:00-0400', tz='America/New_York')},
 'romad': np.float64(2618.1878257288768),
 'total_return': np.float64(0.005068999999999768)}

/snap/core20/current/lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.38' not found (required by /lib64/libduktape.so.207)
Failed to load module: /home/balo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.


In [ ]:
print(datetime(2025, 8, 26).timestamp())

1756162800.0


/snap/core20/current/lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.38' not found (required by /lib64/libduktape.so.207)
Failed to load module: /home/balo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Opening in existing browser session.
